In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [2]:
import numpy as np
import pandas as pd

In [5]:
from scipy.sparse.linalg import svds
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix

In [352]:
ratings_df = pd.read_csv('ratings.csv')
book_df = pd.read_csv('books.csv')

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads
books = db.books

In [1]:
#books.find_one()

In [5]:
ratings = db.ratings
booktags = db.booktags
tags = db.tags

In [6]:
#pip install mongojoin

In [7]:
import pprint

In [8]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 8210, 'Title': 'Den lille havfrue'}
{'Book ID': 7785, 'Title': 'Dawn'}
{'Book ID': 2406, 'Title': 'The Mammoth Hunters'}
{'Book ID': 7306, 'Title': 'Say Cheese and Die!'}
{'Book ID': 815, 'Title': 'No Country for Old Men'}


In [9]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$lookup': {
                        'from': 'ratings',
                        'localField': 'book_id',
                        'foreignField': 'book_id',
                        'as': 'joined'},
                    },
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 4914, 'Title': 'Starless Night (Legacy of the Drow #2)'}
{'Book ID': 4629, 'Title': 'The Expats'}
{'Book ID': 5304, 'Title': 'Midnight'}
{'Book ID': 1180, 'Title': 'The Forty Rules of Love'}
{'Book ID': 8216, 'Title': "Hornet's Nest"}


In [10]:
for book in db.books.find({"book_id":2}):
    print('Book ID: ', book['book_id'])
    print('Title: ', book['original_title'])

Book ID:  2
Title:  Harry Potter and the Philosopher's Stone


In [11]:
def find_book_info(ID):
    for book in db.books.find({'book_id':ID}):
        print (book)

In [13]:
def find_book_rating(ID):
    for book in db.books.find({'book_id':ID}):
        print (book['average_rating'])

In [14]:
def find_book_title(ID):
    for book in db.books.find({'book_id':ID}):
#        print('Book ID: ', book['book_id'])
        print(book['original_title'])

In [486]:
'''
def find_similar_books(itemID, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        if item != itemID:
            recs.append([item+1,np.dot(VT.T[itemID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]
    '''

In [41]:
mat = lil_matrix((53000,10000)) #linked list matrix

In [42]:
mat.shape

(53000, 10000)

In [91]:
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}]) 

In [96]:
rating_cursor

In [92]:
#Test small sample of 5

for rating in rating_cursor:
        u = float(rating['user_id'])
        print(u)
        b = float(rating['book_id'])
        print(b)
        r = float(rating['rating'])
        print(r)
        mat[u, b] = r

11745.0
2979.0
3.0
30118.0
6380.0
4.0
26240.0
2416.0
5.0
4924.0
8402.0
5.0
19777.0
2838.0
2.0


In [83]:
mat.data

array([list([]), list([7.0]), list([8.0]), ..., list([]), list([]),
       list([])], dtype=object)

In [77]:
mat.shape

(53000, 10000)

In [93]:
mat.sum()

19.0

In [95]:
print(mat)

  (4924, 8402)	5.0
  (11745, 2979)	3.0
  (19777, 2838)	2.0
  (26240, 2416)	5.0
  (30118, 6380)	4.0


In [61]:
for u, b, r in zip (u, b, r):
    mat[u,b] = r

In [58]:
int(rating['user_id'])

47524

In [115]:
large_mat = lil_matrix((53425,10001)) #linked list matrix

In [116]:
large_rating_cursor = ratings.find()#aggregate([{'$sample':{'size':100000}}]) 

In [117]:
#Apply to all data

for rating in large_rating_cursor:
        u = float(rating['user_id'])
        b = float(rating['book_id'])
        r = float(rating['rating'])
        large_mat[u, b] = r

In [118]:
large_mat.data.nbytes

427400

In [111]:
from scipy.sparse import csr_matrix

In [121]:
large_mat.tocsr().data.nbytes

47811832

In [122]:
csr_mat = large_mat.tocsr()

In [124]:
csr_mat.shape

(53425, 10001)

In [164]:
U.shape

(53425, 100)

In [165]:
s.shape

(100,)

In [166]:
VT.shape

(100, 10001)

In [167]:
csr_mat.data

array([5., 4., 5., ..., 5., 4., 4.])

In [171]:
csr_mat.shape

(53425, 10001)

In [260]:
U, s, VT = svds(csr_mat,k=175)

In [276]:
U.data.nbytes

74795000

In [277]:
VT.data.nbytes

14001400

In [279]:
import pickle

In [280]:
with open ('U.pickle', 'wb') as to_write:
    pickle.dump(U, to_write)

In [281]:
with open ('VT.pickle', 'wb') as to_write:
    pickle.dump(VT, to_write)

In [261]:
def get_recommendations_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [262]:
def item_similarity(item1,item2):
    return np.dot(VT.T[item1],VT.T[item2])

In [527]:
USERID=20000
num_recom = 11
rec_list = []
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(ix, find_book_title(item[0]), end="")
#    print(item[0])
    find_book_title(item[0])
#    find_book_rating(item[0])

The Girl on the Train
Cien años de soledad
The Glass Castle
The Notebook
My Sister's Keeper
Tuesdays with Morrie
The Giver
An Excellent conceited Tragedie of Romeo and Juliet
Gone Girl
Catching Fire
The Catcher in the Rye


In [327]:
USERID=1
num_recom = 10
rec_list = []
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])

The Hunger Games
Harry Potter and the Philosopher's Stone
Twilight
To Kill a Mockingbird
The Great Gatsby
The Fault in Our Stars
The Hobbit or There and Back Again
The Catcher in the Rye
Angels & Demons
Pride and Prejudice


In [8]:
#find books user has rated a 5

def find_favorites(user_num):

    userread_df = ratings_df[ratings_df.user_id == user_num][ratings_df[ratings_df.user_id == user_num].rating == 5].join(
    book_df, how = 'left', lsuffix = ' ', on = 'book_id')[['user_id','book_id','original_title']]
    
    fav_books = list(userread_df.original_title)
    
    return fav_books

In [9]:
USERID=700
num_recom = 10
rec_list = []

print("User ID:", USERID)
print('\nLast Book Read:\n',np.random.choice(find_favorites(USERID)))
print('\nRecommendations:\n')
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])

User ID: 700


NameError: name 'ratings_df' is not defined

In [443]:
find_favorites(40)

['Siddhartha',
 'The Kite Runner ',
 'The Great Gatsby',
 'Choke',
 'Twilight',
 'The Adventures of Tom Sawyer',
 'A Thousand Splendid Suns',
 'Red Rising',
 'Die Verwandlung',
 'Beneath This Man ',
 'A Walk to Remember',
 'Bared to You',
 'Hija de la fortuna',
 'Jane Eyre',
 'The Lion, the Witch and the Wardrobe',
 'Rise of a Merchant Prince',
 'Fahrenheit 451',
 'Shroud for a Nightingale',
 'Twice Tempted ',
 nan,
 nan,
 '2010: Odyssey Two',
 'Summer Sisters',
 'Queen of the Darkness']

In [512]:
USERID=96
num_recom = 10
rec_list = []
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])

The Stand
Nesnesitelná lehkost bytí
A Light in the Attic
The Light Between Oceans
The Time Traveler's Wife
The Fault in Our Stars
The Giver
The Perks of Being a Wallflower
Divergent
Fahrenheit 451


In [518]:
find_favorites(1)

['The Scorch Trials',
 'The Crying of Lot 49',
 'Divergent',
 'Rant: An Oral Biography of Buster Casey',
 'Outlander',
 'The Giver',
 'The Great Gatsby',
 'Brooklyn',
 'Frankenstein; or, The Modern Prometheus',
 'Circling the Sun',
 "Dirk Gently's Holistic Detective Agency",
 'Water for Elephants',
 'The Bell Jar',
 'Le Comte de Monte-Cristo',
 "One Flew Over the Cuckoo's Nest",
 'Into the Wild',
 'The 4-Hour Workweek: Escape 9-5, Live Anywhere, and Join the New Rich',
 '2010: Odyssey Two',
 'The Novice',
 'Evernight',
 'Complete Harry Potter Boxed Set',
 'Unbroken: A World War II Story of Survival, Resilience, and Redemption']

In [524]:
USERID=200
num_recom = 10
rec_list = []
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])


A Game of Thrones
The Five People You Meet in Heaven
Allegiant
The Time Traveler's Wife
Me Talk Pretty One Day
Marley & Me: Life and Love with the World's Worst Dog
Dark Places
Fifty Shades of Grey
Harry Potter and the Order of the Phoenix


In [525]:
find_favorites(250)

['City of Glass',
 'The Little Engine That Could',
 'The Glass Castle',
 'Sense and Sensibility',
 'Lord of the Flies ',
 'Kiss the Girls',
 'The Lion, the Witch and the Wardrobe',
 'Me Before You',
 "The Wise Man's Fear",
 'The Hunt for Red October',
 'Breaking Dawn',
 'The Great Gatsby',
 'Untamed: A House of Night Novel',
 'The Snowy Day',
 'Twilight',
 'Life As We Knew It',
 "The General's Daughter",
 '90 Minutes in Heaven',
 'City of Bones',
 'Harry Potter and the Chamber of Secrets',
 'A Portrait of the Artist as a Young Man',
 "Harry Potter and the Philosopher's Stone",
 'Of Mice and Men ',
 'The BFG',
 'We3',
 nan,
 'Beneath This Man ',
 'A Walk to Remember',
 'The Dream Thieves',
 'The Road to Character',
 'Aftermath',
 'Harry Potter and the Prisoner of Azkaban',
 'A Stroke of Midnight (Merry Gentry, #4)',
 'The Da Vinci Code',
 'The Farthest Shore',
 'The Maze Runner',
 'Lies and the Lying Liars Who Tell Them: A Fair and Balanced Look at the Right',
 'Iron Kissed',
 'Somethin